In [2]:
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from collections import OrderedDict
from statsmodels.tsa.holtwinters import Holt

In /home/bar/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/bar/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/bar/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/bar/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/bar/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classi

In [3]:
from pathlib import Path

def de_norm(pred, criteria):
    
    df_std = pd.read_pickle(os.path.join(Path(os.getcwd()).parents[0],'df_std.pkl'))
    df_mean = pd.read_pickle(os.path.join(Path(os.getcwd()).parents[0],'df_mean.pkl'))
    std = df_std[criteria]
    mean = df_mean[criteria]
    
    return (pred * std) + mean

# Load Data

In [46]:
# Data to play with

c_type = 'step'
imp = 'mice'
inter = 'nointer'
i= 0

dirpath = os.path.join(Path(os.getcwd()).parents[0], 'datasets', 'train test split')
y_test_30 = pd.read_pickle(os.path.join(dirpath, f"./Y_{c_type}_30_test_{imp}_{inter}_{i}.pkl"))
X_test_30 = pd.read_pickle(os.path.join(dirpath, f"./X_{c_type}_30_test_{imp}_{inter}_{i}.pkl"))

In [76]:
tr_dict = OrderedDict()
X_test_30['y_test_30']  = y_test_30['sys_30']
ids = X_test_30['hadm_id'].unique()

for idx in ids:
    tr_dict[idx] = X_test_30.loc[X_test_30.hadm_id == idx, 'y_test_30'].values

## Trend Utility

According to distance from Holt ES naive prediction.

In [6]:
def trend(tr_dict):
    trends_dict = OrderedDict()
    
    for k,v in tr_dict.items():
        pred_des = [v[0], v[1]]
        for i in range(2,len(v)):
            des = Holt(v[:i]).fit(optimized=True)
            pred_des.append(des.forecast(2)[0])
        trends_dict[k] = np.array(pred_des)
    return trends_dict

# calc trend utility
def g_trend(tr, trend):
    return np.sqrt(np.square(tr - trend))

In [77]:
trends_dict = trend(tr_dict)

/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1439: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1444: RuntimeWarning: invalid value encountered in double_scalars
  aicc = aic + aicc_penalty
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1445: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)


## Normal Range

A naive step function implementation for samples that exceed the thersholds. For Sys thersholds are set to [90,180] to address hypotension and severe hypertension.

In [15]:
def f_norm_naive_sys(tr):
    tr = de_norm(tr, criteria='P_sys_q0.5')
    mask_hyper = (tr > 180)
    mask_hypo = (tr < 90)* -1
    mask = mask_hyper + mask_hypo 
    return mask.astype(int)

In [78]:
un = {}

for k,v in tr_dict.items():
    un[k] = f_norm_naive_sys(v)

In [134]:
# get pred data

file = open(os.path.join(os.getcwd(), 'scatters','30_mice.pkl'), 'rb')
sct = pkl.load(file)
ridge_test_30 = sct['30_bp_step_nointer_mice_ridge'][0]['pred_test']
xbg_test_30 = sct['30_bp_step_nointer_mice_ridge'][0]['pred_test']

ridge_test_30

array([156.03078887, 156.01701018, 151.05291772, ..., 157.80706512,
       157.58100312, 158.01372649])

In [96]:
pred_tr_dict = OrderedDict()
X = X_test_30.copy()
X['y_test_30']  = ridge_test_30

for idx in ids:
    pred_tr_dict[idx] = X.loc[X.hadm_id == idx, 'y_test_30'].values

In [97]:
pred_trends_dict = trend(tr_dict)

/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1439: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1444: RuntimeWarning: invalid value encountered in double_scalars
  aicc = aic + aicc_penalty
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1445: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)


In [98]:
un_pred = {}

for k,v in pred_tr_dict.items():
    un_pred[k] = f_norm_naive_sys(v)

# Normal Range Evaluation Measures

\begin{equation}
\begin{split}
    \frac{1}{n}\sum_{i}\max\left(0, (|f_{norm}\left(y_i\right)| - |f_{norm}\left(y^{pred}_i\right)\right|) + \max(0, sign(f_{norm}(y_i) \cdot sign(f_{norm}(y^{pred}_i))\\
    \frac{1}{n}\sum_{i}\max\left(0, |f_{norm}\left(y^{pred}_i\right)| - |f_{norm}\left(y_i\right)\right|) + \max(0, sign(f_{norm}(y_i)) \cdot sign(f_{norm}(y^{pred}_i))
\end{split}
\end{equation}

In [112]:
def f_norm_eval(dict1, dict2):
    
    sum_norm_total = 0
    for k,v in dict1.items():
        sum_norm = 0
        for i in range(len(dict1[k])):
            sum_norm += max(0,abs(dict1[k][i]) - abs(dict2[k][i])) + max(0, -np.sign(dict1[k][i])*np.sign(dict2[k][i]))
        sum_norm_total += sum_norm / len(dict1[k])
    return sum_norm_total  / len(dict1)

In [113]:

print(f'missing important: {f_norm_eval(un, un_pred)}')
print(f'false alarm: {f_norm_eval(un_pred, un)}')


missing important: 0.0
false alarm: 0.8480776341104778


# Trend Based Evaluation


\begin{equation}
\begin{split}
    W_{g}(y_i)\cdot |y_i - y^{pred}_i|\\
    W_{g}(y^{pred}_i)\cdot |y_i - y^{pred}_i|
\end{split}
\end{equation}


In [115]:
# Calc Weights

values = [v for k,v in trends_dict.items()]
u_max = max(np.hstack(values))
print(f'u_max: {u_max}')

values = [v for k,v in pred_trends_dict.items()]
u_max_pred = max(np.hstack(values))
print(f'u_max_pred: {u_max_pred}')


# calculate weights
weights = {k:v/u_max for k,v in trends_dict.items()}
weights_pred = {k:v/u_max for k,v in pred_trends_dict.items()}

u_max: 1.6043447911875044
u_max_pred: 1.6043447911875044


In [130]:
w = weights

def w_trend(dict1, dict2, w):
    total_sum_trend = 0
    for k,v in w.items():
        sum_trend = 0
        for i in range(len(dict1[k])):
            sum_trend += w[k][i]*abs(dict1[k][i] - dict2[k][i]) / len(dict1[k])
            
        sum_trend += w[k][i]*abs(dict1[k][i] - dict2[k][i])
    return np.mean(sum_trend) 


In [131]:
print(f'missing important: {w_trend(un, un_pred, weights)}')
print(f'false alarm: {w_trend(un_pred, un, weights_pred)}')

missing important: 0.41214923001573267
false alarm: 0.41214923001573267


In [132]:
pref_dict= {'Normal Range Deviation Miss':f_norm_eval(un, un_pred),\
            'Normal Range Deviation False Alarm':f_norm_eval(un_pred, un),\
           'Trend Deviation Miss': w_trend(un, un_pred, weights),\
            'Trend Deviation False Alarm': w_trend(un_pred, un, weights_pred)}

In [133]:
pd.DataFrame(pref_dict, index=[1]).round(3)

,Normal Range Deviation Miss,Normal Range Deviation False Alarm,Trend Deviation Miss,Trend Deviation False Alarm
1,0.0,0.848,0.412,0.412
